In [3]:
# Import Dependencies
import pandas as pd
from census import Census
from config import census_key
import cpi # this will be used to determine the cost of everything in today's dollars. cpi.inflate(amount,year of amount)
#install cpi with pip install cpi on command line
cpi.update()
low_memory=False

ModuleNotFoundError: No module named 'census'

In [4]:
# Import CSVs 
filepath = "Resources/"
kent_zipcodes = pd.read_csv(filepath+"kent_zipcodes.csv",squeeze=True,header=None)
oakland_zipcodes = pd.read_csv(filepath+"oakland_zipcodes.csv",squeeze=True,header=None)
saginaw_zipcodes = pd.read_csv(filepath+"saginaw_zipcodes.csv",squeeze=True,header=None)



In [5]:
# Create dataframe for each imported CSV and add a column with the corresponding county
kent_zipcodes = pd.DataFrame(kent_zipcodes)
kent_zipcodes["County"] = "Kent"
oakland_zipcodes = pd.DataFrame(oakland_zipcodes)
oakland_zipcodes["County"] = "Oakland"
saginaw_zipcodes = pd.DataFrame(saginaw_zipcodes)
saginaw_zipcodes["County"] = "Saginaw"
# Merge zipcode dataframes and add a column name for Zipcode
query_zipcodes =pd.DataFrame( pd.merge(kent_zipcodes,oakland_zipcodes,how="outer"))
query_zipcodes = pd.DataFrame(pd.merge(query_zipcodes,saginaw_zipcodes,how="outer"))
query_zipcodes.columns = ["Zipcode","County"]
query_zipcodes = query_zipcodes.astype({"Zipcode":object})

In [6]:
query_zipcodes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172 entries, 0 to 171
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Zipcode  172 non-null    object
 1   County   172 non-null    object
dtypes: object(2)
memory usage: 4.0+ KB


In [5]:
# Create dataframe to add census data to. Needed to put in starter values for it to function properly later. Will remove further down.
census_df = pd.DataFrame({"Year":[0],
                          "Zipcode":["0"],
                          "Total Population":[0],
                          "White Population":[0],
                          "Black Population":[0],
                          "Employed Count":[0],
                          "Unemployed Count":[0],
                          "Poverty Count":[0],
                          "Poverty Count - White":[0],
                          "Poverty Count - Black":[0],
                          "Median Income":[0],
                          "Median Rent":[0],
                          "Median Home Value":[0]})

# Get data from API for Census
census_df = census_df.astype({"Year":int,"Zipcode":object})

# For loop to iterate through the desired 10-year period
for year in [2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]: 

    c = Census(census_key,year) 
    census_data = c.acs5.get(("NAME", "B19013_001E", "B23025_004E", "B23025_005E", "B17001_002E", 
            "B17001A_002E","B17001B_002E", "B01003_001E",
            "B02001_002E", "B02001_003E","B25058_001E", "B25077_001E"
             ), {"for": "zip code tabulation area:*"})
    census_raw_df = pd.DataFrame(census_data)
    
# Add the year to the year column so each iteration can be distinguished
    year_count = len(census_data)
    year_series = pd.Series([year for x in range(year_count)])    
    census_raw_df["Year"] = year_series
    
# Rename Columns
    census_raw_df = census_raw_df.rename(columns = {
            "B19013_001E" : "Median Income", 
            "B23025_004E" : "Employed Count", 
            "B23025_005E" : "Unemployed Count", 
            "B17001_002E" : "Poverty Count", 
            "B17001A_002E" : "Poverty Count - White" , 
            "B17001B_002E" : "Poverty Count - Black" , 
            "B01003_001E" : "Total Population", 
            "B02001_002E" : "White Population", 
            "B02001_003E" : "Black Population", 
            "B25058_001E" : "Median Rent", 
            "B25077_001E" : "Median Home Value", 
            "NAME" : "Name",
            "zip code tabulation area": "Zipcode"})
    
# Recast all variables in the output dataframe to the correct type 
    census_raw_df = census_raw_df.astype({"Year":int,"Zipcode":object,"Median Income":float,"Employed Count":float,"Unemployed Count":float,"Poverty Count":float,
             "Poverty Count - White":float, "Poverty Count - Black":float, "Total Population":float,
             "White Population":float,"Black Population":float, "Median Rent":float,"Median Home Value":float})
    
# Recast the Zipcode in census_df to object. Was running into issues with the zipcode repeatedly trying to cast as a float.
    census_df = census_df.astype({"Zipcode":object})
    
# Merge the new raw df into the finished df. Use outer to maintain all values
    census_df = pd.merge(census_df,census_raw_df,how="outer")

In [6]:
for i in query_zipcodes.index:
    query_zipcodes.at[i,"Zipcode"] = str(query_zipcodes.loc[i,"Zipcode"])
query_zipcodes

,Zipcode,County
0,49508,Kent
1,49504,Kent
2,49507,Kent
3,49503,Kent
4,49341,Kent
...,...,...
167,48605,Saginaw
168,48606,Saginaw
169,48608,Saginaw
170,48663,Saginaw


In [9]:
# Merge Dataframes so only those zipcodes we want are there. Also adds County 
census_df = pd.merge(census_df,query_zipcodes,how="inner", on="Zipcode")

,Year,Zipcode,Total Population,White Population,Black Population,Employed Count,Unemployed Count,Poverty Count,Poverty Count - White,Poverty Count - Black,Median Income,Median Rent,Median Home Value,County
0,2012,48009,20153,18622,663,10031.0,489.0,797.0,767.0,0.0,100789.0,993.0,339600.0,Oakland
1,2013,48009,20241,18620,765,10138.0,553.0,704.0,669.0,0.0,98750.0,1070.0,347400.0,Oakland
2,2015,48009,20489,18864,587,10669.0,407.0,827.0,730.0,37.0,108135.0,1121.0,410700.0,Oakland
3,2017,48009,20836,19063,580,10795.0,393.0,914.0,726.0,92.0,114537.0,1223.0,467700.0,Oakland
4,2020,48009,21277,19011,688,10729.0,362.0,1001.0,790.0,114.0,126646.0,1498.0,538500.0,Oakland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,2017,49548,32694,23495,3522,16310.0,963.0,6656.0,4210.0,932.0,45534.0,695.0,82200.0,Kent
1246,2012,49548,30155,23134,3109,13743.0,1895.0,6416.0,4296.0,1162.0,39926.0,622.0,89700.0,Kent
1247,2015,49548,30234,22081,3032,14715.0,1308.0,6056.0,3203.0,996.0,40895.0,648.0,78800.0,Kent
1248,2020,49548,33608,23289,3177,17354.0,722.0,5203.0,3357.0,749.0,50763.0,768.0,106800.0,Kent


In [1]:
# Remove the name column, which is useless here, and the state column, which only functioned for some years
census_df = census_df.drop(columns = ["Name","state"])

#Remove the sample data added to initialize the dataframe
census_df = census_df.loc[census_df["Year"] != 0]

NameError: name 'census_df' is not defined

In [12]:
census_df.sort_values("Median Income")

,Year,Zipcode,Total Population,White Population,Black Population,Employed Count,Unemployed Count,Poverty Count,Poverty Count - White,Poverty Count - Black,Median Income,Median Rent,Median Home Value,County
886,2017,48724,458,398,60,217.0,0.0,136.0,136.0,0.0,-666666666.0,-666666666.0,70900.0,Saginaw
880,2018,48724,484,423,61,227.0,0.0,145.0,145.0,0.0,-666666666.0,-666666666.0,71900.0,Saginaw
881,2019,48724,544,470,74,241.0,20.0,227.0,227.0,0.0,-666666666.0,-666666666.0,70000.0,Saginaw
769,2011,48607,1813,355,1373,361.0,169.0,873.0,212.0,648.0,15552.0,376.0,62200.0,Saginaw
766,2012,48607,1628,286,1255,365.0,165.0,761.0,161.0,589.0,17695.0,390.0,63100.0,Saginaw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,2018,48306,27663,23587,858,13366.0,412.0,469.0,411.0,4.0,147303.0,853.0,386500.0,Oakland
568,2015,48374,15316,10717,594,7809.0,324.0,999.0,571.0,87.0,147536.0,1640.0,367800.0,Oakland
236,2020,48301,15024,12160,841,7182.0,357.0,421.0,365.0,20.0,147585.0,1443.0,544500.0,Oakland
80,2020,48070,6303,5869,62,3500.0,65.0,92.0,92.0,0.0,150114.0,1754.0,372200.0,Oakland


In [20]:
census_df = census_df.loc[(census_df["Zipcode"]!= "48724") & (census_df["Zipcode"]!= "48370") & (census_df["Zipcode"]!= "48380")]

In [21]:
census_df.sort_values("Median Rent").head(20)

,Year,Zipcode,Total Population,White Population,Black Population,Employed Count,Unemployed Count,Poverty Count,Poverty Count - White,Poverty Count - Black,Median Income,Median Rent,Median Home Value,County
674,2012,48417,2830,2785,0,1187.0,209.0,426.0,399.0,0.0,42361.0,290.0,106000.0,Saginaw
911,2012,48807,800,777,0,339.0,51.0,70.0,67.0,0.0,49135.0,300.0,116800.0,Saginaw
675,2013,48417,2741,2699,0,1075.0,180.0,311.0,285.0,0.0,46382.0,308.0,103000.0,Saginaw
763,2015,48607,1305,215,968,452.0,54.0,542.0,113.0,385.0,20994.0,338.0,38800.0,Saginaw
768,2013,48607,1392,294,949,445.0,119.0,604.0,179.0,409.0,21302.0,368.0,45200.0,Saginaw
769,2011,48607,1813,355,1373,361.0,169.0,873.0,212.0,648.0,15552.0,376.0,62200.0,Saginaw
761,2016,48607,1236,245,815,357.0,49.0,540.0,105.0,378.0,20710.0,378.0,40700.0,Saginaw
785,2013,48614,1375,1357,1,517.0,65.0,310.0,309.0,1.0,48942.0,378.0,104300.0,Saginaw
765,2014,48607,1584,286,1165,522.0,132.0,821.0,163.0,622.0,20833.0,382.0,43900.0,Saginaw
766,2012,48607,1628,286,1255,365.0,165.0,761.0,161.0,589.0,17695.0,390.0,63100.0,Saginaw


In [24]:
# Adjust financial columns to account for inflation. All values go to current-day inflation (2023)
for i in census_df.index:
    census_df.at[i,"Adjusted Median Income"] = cpi.inflate((census_df.at[i,"Median Income"]),(census_df.at[i,"Year"]))
    census_df.at[i,"Adjusted Median Rent"] = cpi.inflate((census_df.at[i,"Median Rent"]),(census_df.at[i,"Year"]))
    census_df.at[i,"Adjusted Median Home Value"] = cpi.inflate((census_df.at[i,"Median Home Value"]),(census_df.at[i,"Year"]))


In [26]:
census_df.to_csv("output_data/census.csv",index_label="datapoint")